<a href="https://colab.research.google.com/github/daikichiba9511/study_log/blob/master/pytorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pytorchチュートリアル 



In [0]:
import torch

In [63]:
#初期化された(5,3)行列をTensor型で宣言
x=torch.Tensor(5,3)
x

tensor([[1.1626e-36, 0.0000e+00, 4.4842e-44],
        [0.0000e+00,        nan, 4.4842e-44],
        [6.6015e-07, 1.3163e+22, 3.2943e-09],
        [6.7707e+22, 3.3462e+21, 2.6881e-06],
        [1.6595e-07, 1.0440e+21, 1.5046e-36]])

In [64]:
#乱数で初期化したものをTensor型で宣言する
y=torch.rand(5,3)
y

tensor([[0.4400, 0.0593, 0.3748],
        [0.5465, 0.0731, 0.9827],
        [0.7845, 0.4821, 0.5605],
        [0.7022, 0.6427, 0.5139],
        [0.7055, 0.1519, 0.1356]])

In [65]:
#サイズの確認
x.size()

torch.Size([5, 3])

In [66]:
x+y

tensor([[4.4003e-01, 5.9301e-02, 3.7483e-01],
        [5.4652e-01,        nan, 9.8265e-01],
        [7.8449e-01, 1.3163e+22, 5.6048e-01],
        [6.7707e+22, 3.3462e+21, 5.1386e-01],
        [7.0552e-01, 1.0440e+21, 1.3563e-01]])

In [67]:
#加算
result=torch.Tensor(5,3) #make empty (5,3) Tensor
torch.add(x,y,out=result)
result

tensor([[4.4003e-01, 5.9301e-02, 3.7483e-01],
        [5.4652e-01,        nan, 9.8265e-01],
        [7.8449e-01, 1.3163e+22, 5.6048e-01],
        [6.7707e+22, 3.3462e+21, 5.1386e-01],
        [7.0552e-01, 1.0440e+21, 1.3563e-01]])

In [68]:
y.add_(x)  #"add_(x)" -> y=y+x

tensor([[4.4003e-01, 5.9301e-02, 3.7483e-01],
        [5.4652e-01,        nan, 9.8265e-01],
        [7.8449e-01, 1.3163e+22, 5.6048e-01],
        [6.7707e+22, 3.3462e+21, 5.1386e-01],
        [7.0552e-01, 1.0440e+21, 1.3563e-01]])

In [0]:
from torch.autograd import Variable

In [70]:
#try to make variable
x=Variable(torch.ones(2,2),requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [71]:
#add
y=x+2
print(y)
#mul
z=x*4
z

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


tensor([[4., 4.],
        [4., 4.]], grad_fn=<MulBackward0>)

In [72]:
#演算後に生成されたVariableには属性grad_fnがふよされる
y.grad_fn

In [73]:
z2=y*y*3
out=z2.mean()
print(z2,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


# 勾配計算
 
*逆伝播の計算を行う* 

In [0]:
out.backward()

In [75]:
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

In [76]:
x=torch.randn(3)
x=Variable(x,requires_grad=True)

y=x*2
while y.data.norm()<1000:
  y=y*2
  
print(y)

tensor([ 669.7197,   97.5865, 1031.4189], grad_fn=<MulBackward0>)


In [77]:
gradients = torch.FloatTensor([0.1,1.0,0.0001])
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [0]:
from torch import Tensor

In [0]:
#手書き文字を分類するNN
#パッケージのインポート
import torch
from torch.autograd  import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  
    def __init__(self):
        super(Net,self).__init__()
    #1 input image channel, 6 output chabbels, 5*5 square convolution
    #kernel
        self.conv1=nn.Conv2d(1,6,5) #(input,channels output )
        self.conv2=nn.Conv2d(6,16,5)
    
    #an affine operattion: y=Wx+b　,full connection
        self.fc1=nn.Linear(16*5*5,120) #(input,output)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    
    def forward(self,x): 
    #max pooling over a (2,2) window
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
    #if the size is a square you can only specify a single number
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
  
    def num_flat_features(self,x):
        size=x.size()[1:] #all dimensions except the batch dimension
        num_features=1
        for s in size:
            num_features *= s
        return num_features
    

  
    

In [83]:
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward関数を定義するだけで、backward関数は自動的に定義できる。

backward関数：勾配計算

これがautogradの便利なところ


### 学習可能なパラメタはnet.parameters()で確認できる


In [84]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) #conv1.weight

10
torch.Size([6, 1, 5, 5])


LeNetの入力は32*32だから、mnistデータセットで
このネットワークを使うなら32*32にリサイズする必要

全てのパラメータの勾配情報を溜め込んでいるバッファーを０にし、ランダムな勾配を逆伝播させる


In [86]:
input =Tensor(torch.randn(1,1,32,32))
out = net(input)
print(out)

tensor([[ 0.0559, -0.0782,  0.0175, -0.1577, -0.0046, -0.1862,  0.0019,  0.0368,
         -0.0388, -0.0667]], grad_fn=<AddmmBackward>)


**point:**  torch.nnはミニバッチのみサポート。単体のデータを利用できない

例えば、nn.Conv2dは４次元テンソル

`` nSample × nChannels × Height × Width ``

を引数としてとる。

単体のサンプルを入力するときは、input.unsqueeze(0)を使うことでバッチの次元をごまかせる



## ここまでのまとめ

- `torch.Tensor` : 多次元配列
- `Variable` : は現在では、 Tensorにmergeされた。autogradの中にbackward等のメソッドなど。
詳しくは公式ドキュメント参照
- `nn.Module` : NNのモジュール。パラメタをカプセル化。パラメタをGPUに載せたり、出力したり、ロードしたりする便利な機能付き
- `nn.Parameter` : Variableの一種で `Module` :を定義したとき、自動的に登録される。
- `autograd.Function` : autogradの演算における順伝播の定義を実装したパッケージ。全てのVariableの演算は少なくとも１つのFunctionノードを作る。そしてこのノードはVariableを生成した関数と結び付けられる。そしてこのノードが履歴を再現する

ここまで、

1. NNを定義する

2. 入力の処理とbackwardの呼び出し

について見てきた。

次は、

- 損失の計算

- networkの更新

について見ていく。